In [79]:
%load_ext autoreload
%load_ext line_profiler
%autoreload 2

import numpy as np
from mpl_toolkits import mplot3d
import matplotlib
import matplotlib.pyplot as plt
import scipy.optimize as optimize

from LimitedCommitmentModel import LimitedCommitmentModelClass

# plot style
linestyles = ['-','--','-.',':',':']
markers = ['o','s','D','*','P']
linewidth = 2
font_size = 17
font = {'size':font_size}
matplotlib.rc('font', **font)

plt.rcParams.update({'figure.max_open_warning': 0,'text.usetex': False})
path = 'output/'

SAVE = False

# c++ settings
do_compile = True
threads = 7

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


# Solve the model

In [80]:
# compile c++ files
T = 10
specs = {  
    'model 3, limited':{'latexname':'limited', 'par':{'sigma_love':0.1,'T':T,'threads':threads,'bargaining':1}},
}


# solve different models
models = {}
for m,(name,spec) in enumerate(specs.items()):
    print(f'{name} loading...',end='')
    
    # setup model
    models[name] = LimitedCommitmentModelClass(name=name,par=spec['par'])
    models[name].spec = spec

    compile_now = True if do_compile & (m==0) else False
    models[name].link_to_cpp(force_compile=compile_now)
    
    print(' solving...')
    %time models[name].solve() 

model 3, limited loading... solving...
CPU times: total: 1h 7min 52s
Wall time: 11min 21s


In [83]:

model = models['model 3, limited']
model.sim.init_A[:] = 1.0 
model.simulate()
model.sim.labor_w
#model.sol.labor_w_couple

[[[[[[1.00000000e+00 1.00000000e+00 1.00000000e+00 ... 1.00000000e+00
      1.00000000e+00 1.00000000e+00]
     [1.00000000e+00 1.00000000e+00 1.00000000e+00 ... 1.00000000e+00
      1.00000000e+00 1.00000000e+00]
     [1.00000000e+00 1.00000000e+00 1.00000000e+00 ... 1.00000000e+00
      1.00000000e+00 1.00000000e+00]
     ...
     [1.00000000e+00 1.00000000e+00 1.00000000e+00 ... 1.00000000e+00
      1.00000000e+00 1.00000000e+00]
     [1.00000000e+00 1.00000000e+00 1.00000000e+00 ... 1.00000000e+00
      1.00000000e+00 1.00000000e+00]
     [1.00000000e+00 1.00000000e+00 1.00000000e+00 ... 1.00000000e+00
      1.00000000e+00            nan]]

    [[1.00000000e+00 1.00000000e+00 1.00000000e+00 ... 1.00000000e+00
      1.00000000e+00 1.00000000e+00]
     [1.00000000e+00 1.00000000e+00 1.00000000e+00 ... 1.00000000e+00
      1.00000000e+00 1.00000000e+00]
     [1.00000000e+00 1.00000000e+00 1.00000000e+00 ... 1.00000000e+00
      1.00000000e+00 1.00000000e+00]
     ...
     [1.00000000e

array([[1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       ...,
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.]])

# Estimate the model

In [50]:
#Estimate the model one time
model = models['model 3']

model.sim.init_A[:] = 1.0 # temp just to check that it works, if they have too little initial saving they will just consume everything, and saving is therefore close to zero, and the model cannot be estimated

np.random.seed(model.par.seed)
data = Estimate.create_data(model,start_p = 1, end_p = 4, to_xl = True)
data_reg = Estimate.aux_est(data,print_reg=True)
data_reg, Wald_FC, Wald_NC = Estimate.main_est(data_reg,print_reg=True) 

c:\Users\czk481\Documents\PhD\projects\household_bargening\LimitedTest\LimitedCommitmentTest\Estimate.py:29: RuntimeWarning: overflow encountered in exp
  wage_w      =  np.exp(model.par.wage_const_w +model.par.wage_K_w* model.sim.Kw)
c:\Users\czk481\Documents\PhD\projects\household_bargening\LimitedTest\LimitedCommitmentTest\Estimate.py:30: RuntimeWarning: overflow encountered in multiply
  y_w         =  wage_w*model.sim.labor_w
c:\Users\czk481\Documents\PhD\projects\household_bargening\LimitedTest\LimitedCommitmentTest\Estimate.py:31: RuntimeWarning: overflow encountered in exp
  wage_m      =  np.exp(model.par.wage_const_m+model.par.wage_K_m* model.sim.Km)
c:\Users\czk481\Documents\PhD\projects\household_bargening\LimitedTest\LimitedCommitmentTest\Estimate.py:32: RuntimeWarning: overflow encountered in multiply
  y_m         =  wage_m*model.sim.labor_m
c:\Users\czk481\Documents\PhD\projects\household_bargening\LimitedTest\LimitedCommitmentTest\Estimate.py:32: RuntimeWarning: invali

Residuals from hours equation
                                 OLS Regression Results                                
Dep. Variable:        delta_log_Labor   R-squared (uncentered):                   0.025
Model:                            OLS   Adj. R-squared (uncentered):              0.025
Method:                 Least Squares   F-statistic:                              2272.
Date:                Thu, 18 Jan 2024   Prob (F-statistic):                        0.00
Time:                        18:10:39   Log-Likelihood:                     -4.2206e+05
No. Observations:              270584   AIC:                                  8.441e+05
Df Residuals:                  270581   BIC:                                  8.442e+05
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
                       coef    std err          t      P>|t|      [0.025      0.975]
-----

c:\Users\czk481\Anaconda3\lib\site-packages\statsmodels\base\model.py:1906: FutureWarning: The behavior of wald_test will change after 0.14 to returning scalar test statistic values. To get the future behavior now, set scalar to True. To silence this message while retaining the legacy behavior, set scalar to False.
  warnings.warn(
